# Joergensen and Raubuch (2002)

## DEB model calibration

### Importing functions

In [1]:
import numpy as np
import pandas as pd
from scipy.integrate import odeint
from scipy.optimize import dual_annealing
from scipy.optimize import differential_evolution

### This is the DEB model

In [2]:
def DEBmodel (y, t, pars):
    #define initial pools (eu is assumed to be zero)
    S=y[0];    e=y[1];    X1=y[2];     CO2=y[3];
    
    #define parameters
    yA=pars[0]; 
    Km=pars[1];     
    v=pars[2];
    m=pars[3]; 
    g=pars[4]; 
    #k=pars[5];
    ce=pars[5];
    MX1=ce/4;
    
    #Scaling function for substrate uptake
    f=S/(Km+S)
    #Fluxes
    uptake=(v*ce/yA)*X1*f
    growth = (v*e-m*g)/(e + g)
    
    #Define derivatives
    dSdt = -uptake
    dedt = v*(f - e)
    dX1dt = growth*X1 #- k*X1
    dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1 
           
    return dSdt, dedt, dX1dt, dCO2dt;

### The function below uses the output from DEBmodel to convert biomass pools $e$ and $X_{1}$ to microbial biomass ($B$) and measured biomass proxy-parameters ATP and Flush (after chloroform fumigation) 

In [3]:
def calcDEB (model, pars, t, y0):
    #model parameters
    ##yA, Km, v, m, g, ce
    pars_model=pars[0:6]
    #conversion factors
    ##ce, nX1, te, tX1
    conversions=pars[5:9]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate biomass (B), total ATP, and flush (Flush)
    B=(conversions[0]/4 + conversions[0]*y[:, 1])*y[:, 2]
    Flush = (conversions[1]/4 + y[:, 1])/(0.25 + y[:, 1])*B
    ATP = (conversions[3]/4 + conversions[2]*y[:, 1])/(0.25 + y[:, 1])*B
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(d.Time),1),#glucose
                           #y[:, 3].reshape(len(d.Time[0:18]),1),#CO2
                           Flush.reshape(len(d.Time),1),
                           ATP.reshape(len(d.Time),1)), axis=1)

    return yhat

### Objective function is defined

In [4]:
def obj_funDEB (x):
    #define parameters
    ##yA, Km, v, m, g, ce, nX1, te, tX1
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
            
    e_i = 0.25*((d.ATPinit[0]/d.Cmicinit[0])*pars[6] - pars[8])/(pars[7] - (d.ATPinit[0]/d.Cmicinit[0]))
    X1_i = d.Cmicinit[0]/(0.25*pars[6] + e_i)/pars[5]
    
    y0 = np.array([S_i, e_i, X1_i, 0])
    #times
    t = d.Time

    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    
     
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        #np.array([d.CO2cumul]).reshape(len(d.Time),1),
                        np.array([d.Cmic]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.ATP]).reshape(len(d.Time),1)),
                     axis=1)

    #weights
    weights=np.concatenate((np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd(d.CO2cumul[0:18]).repeat(len(d.Time[0:18])).reshape(len(d.Time[0:18]),1),
                            np.nanstd((d.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.ATP)).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    out=np.nansum(((yhat_full-obs)/weights)**2)

    return out

### Goodness of fit is calculated

In [5]:
def goodnessDEB (x):
    #define parameters
    ##yA, Km, v, m, g, ce, nX1, te
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
            
    e_i = 0.25*((d.ATPinit[0]/d.Cmicinit[0])*pars[6] - pars[8])/(pars[7] - (d.ATPinit[0]/d.Cmicinit[0]))
    X1_i = d.Cmicinit[0]/(0.25*pars[6] + e_i)/pars[5]
    
    y0 = np.array([S_i, e_i, X1_i, 0])
    #times
    t = d.Time

    #model simulations
    yhat_full = calcDEB(DEBmodel, pars, t, y0)
    
    #Standardize the simulations
    ##means
    Smeans=np.concatenate((np.nanmean(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Sstd=np.concatenate((np.nanstd(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
     

    #observations
    obs=np.concatenate((np.array([d2.S]).reshape(len(d.Time),1),
                        #np.array([d.CO2cumul]).reshape(len(d.Time),1),
                        np.array([d2.Cmic]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d2.ATP]).reshape(len(d.Time),1)),
                     axis=1)
    #Standardize the observations
    ##means
    Omeans=np.concatenate((np.nanmean(d2.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.CO2cumul[0:18]).repeat(18).reshape(18,1),
                            np.nanmean((d2.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d2.ATP).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std    
    Ostd=np.concatenate((np.nanstd(d2.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd(d.CO2cumul[0:18]).repeat(18).reshape(18,1),
                            np.nanstd((d2.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d2.ATP).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
        
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    #Normalized residual sum of squares 
    Fnorm = np.nansum((((obs-Omeans)/Ostd)-((yhat_full-Smeans)/Sstd))**2)
    
    out = np.array([R2, ll, AIC, Fnorm])

    return out

In [6]:
def predDEB (model, pars, t, y0):
    #model parameters
    ##yA, Km, v, m, g, ce
    pars_model=pars[0:6]
    #conversion factors
    ##ce, nX1, ne
    conversions=pars[5:9]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate biomass (B), total ATP, and flush (Flush)
    B=(conversions[0]/4 + conversions[0]*y[:, 1])*y[:, 2]
    Flush = (conversions[1]/4 + y[:, 1])/(0.25 + y[:, 1])*B
    ATP = (conversions[3]/4 + conversions[2]*y[:, 1])/(0.25 + y[:, 1])*B
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(np.arange(0, 8.05, 0.05)),1),#glucose
                           y[:, 1].reshape(len(np.arange(0, 8.05, 0.05)),1),#e
                           y[:, 2].reshape(len(np.arange(0, 8.05, 0.05)),1),#X1
                           B.reshape(len(np.arange(0, 8.05, 0.05)),1),#biomass
                           #y[:, 3].reshape(len(np.arange(0, 9.05, 0.05)),1),#CO2
                           Flush.reshape(len(np.arange(0, 8.05, 0.05)),1),
                           ATP.reshape(len(np.arange(0, 8.05, 0.05)),1)), axis=1)

    return yhat

In [7]:
#reading the data
d = pd.read_csv('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/SoilMBVariabilityData/Joergensen2002.csv', sep=',')
d2 = d
#remove one outlier 
d.Cmic[5] = np.nan
d.ATP[5] = np.nan
print(d)

                          Study  Soil Substrate  Clay   pH  Ctot      Ntot  \
0  Joergensen and Raubuch, 2002   NaN   Glucose     9  7.1   1.4  0.127273   
1  Joergensen and Raubuch, 2002   NaN   Glucose     9  7.1   1.4  0.127273   
2  Joergensen and Raubuch, 2002   NaN   Glucose     9  7.1   1.4  0.127273   
3  Joergensen and Raubuch, 2002   NaN   Glucose     9  7.1   1.4  0.127273   
4  Joergensen and Raubuch, 2002   NaN   Glucose     9  7.1   1.4  0.127273   
5  Joergensen and Raubuch, 2002   NaN   Glucose     9  7.1   1.4  0.127273   
6  Joergensen and Raubuch, 2002   NaN   Glucose     9  7.1   1.4  0.127273   

   Time  Cmicinit  Nmicinit  ATPinit       Sinit       Cmic      Nmic  \
0  0.00  5.695254  1.194861   0.0114  133.217878   5.695254  1.194861   
1  0.25  5.695254  1.194861   0.0114  133.217878  13.263947  1.310493   
2  0.50  5.695254  1.194861   0.0114  133.217878  17.348043  1.773019   
3  1.00  5.695254  1.194861   0.0114  133.217878  17.572856  1.580300   
4  2.00  5

<ipython-input-7-34aae0666234>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d.Cmic[5] = np.nan
<ipython-input-7-34aae0666234>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d.ATP[5] = np.nan


In [8]:
j_debpars=dual_annealing(obj_funDEB, [(0.05, 1), #yA
                                             (10, 10000), #Km
                                             (0.001, 20), #v
                                             (1e-15, 0.1), #m
                                             (0.1, 5), #g
                                             #(1e-12, 0.1), #k
                                             (0.1, 10), #ce
                                             (0, 1), #nX1
                                             (0, 1), #tX1
                                             (0, 1)], maxiter = 10000) #te
                                             #(0, 1)]) #iX1

/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
<ipython-input-4-4433fcaacdba>:10: RuntimeWarning: divide by zero encountered in double_scalars
  X1_i = d.Cmicinit[0]/(0.25*pars[6] + e_i)/pars[5]
<ipython-input-2-a3081a537b9a>:25: RuntimeWarning: invalid value encountered in double_scalars
  dCO2dt = uptake*(1 - yA) + ce*(X1*e*(v-growth)) - growth*X1*MX1
/usr/local/lib/python3.8/dist-packages/scipy/integrate/odepack.py:247: ODEintWarning: Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
<ipython-input-3-b416a855a223>:14: RuntimeWarning: invalid value encountered in multiply
  Flush = (conversions[1]/4 + y[:, 1])/(0.25 + y[:, 1])*B
<ipython-input-3-b416a855a223>:15: RuntimeWarning: invalid value encou

In [9]:
print(j_debpars)
print(goodnessDEB(j_debpars.x))

     fun: 0.6900002077441969
 message: ['Maximum number of iteration reached']
    nfev: 412289
    nhev: 0
     nit: 10000
    njev: 23228
  status: 0
 success: True
       x: array([3.96309549e-01, 2.62447430e+02, 4.23386740e+00, 1.79431894e-02,
       1.59708364e+00, 8.63793519e+00, 1.00000000e+00, 0.00000000e+00,
       1.63653490e-03])
[ 0.9795507  -0.19426832 18.38853664  0.95267755]


In [ ]:
j_debparsDE=differential_evolution(obj_funDEB, [(0.05, 1), #yA
                                             (10, 10000), #Km
                                             (0.001, 20), #v
                                             (1e-15, 0.1), #m
                                             (0.1, 5), #g
                                             #(1e-12, 0.1), #k
                                             (0.1, 10), #ce
                                             (0, 1), #nX1
                                             (0, 1), #te
                                             (0, 1)], maxiter = 30000) #tX1

In [ ]:
print(j_debparsDE)
print(goodnessDEB(j_debparsDE.x))

In [ ]:
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Joergensen2002Pars.csv', j_debparsDE.x.reshape(1,9), delimiter=",")

## Solution over time for visualization in R

In [ ]:
#initial conditions
S_i = d.Sinit[0]
    
e_i = 0.25*((d.ATPinit[0]/d.Cmicinit[0])*j_debparsDE.x[6] - j_debparsDE.x[8])/(j_debparsDE.x[7] - (d.ATPinit[0]/d.Cmicinit[0]))
X1_i = d.Cmicinit[0]/(0.25*j_debparsDE.x[6] + e_i)/j_debparsDE.x[5]
    
y0 = np.array([S_i, e_i, X1_i, 0])

#times
t = np.arange(0, 8.05, 0.05)

#model simulations
Joergensen2002Pred = predDEB(DEBmodel, j_debparsDE.x, t, y0)
   
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Joergensen2002Pred.csv', Joergensen2002Pred, delimiter=",")

# Monod and Pirt models are fitted for comparison to DEB

## Monod model
Model as well as supplementary functions are defined below

In [ ]:
def Mmodel (y, t, pars):
    #define initial pools
    S=y[0];    B=y[1];    CO2=y[2];
    
    #define parameters
    v=pars[0]; 
    Km=pars[1];     
    CUE=pars[2];
    k=pars[3];
        
    #Fluxes
    uptake = v*S*B/(S + Km)
    growth = uptake*CUE
    respiration = uptake*(1 - CUE)
    death = B*k
    
    #Define derivatives
    dSdt = -uptake
    dBdt = growth - death
    dCO2dt = respiration
           
    return dSdt, dBdt, dCO2dt;

In [ ]:
def calcM (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, k
    pars_model=pars[0:4]
    #conversion factors
    ##kec, katp
    conversions=pars[4:6]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total ATP, and flush (Flush)
    Flush = conversions[0]*y[:, 1]
    ATP = conversions[1]*y[:, 1]
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(d.Time),1),#glucose
                           #y[:, 2].reshape(len(d.Time),1),#CO2
                           Flush.reshape(len(d.Time),1),
                           ATP.reshape(len(d.Time),1)), axis=1)

    return yhat

In [ ]:
def obj_funM (x):
    #define parameters
    ##v, Km, CUE, k, kec, katp
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
       
    B_i = d.Cmicinit[0]/pars[4]
    
    y0 = np.array([S_i, B_i, 0])
    
    #times
    t = d.Time

    #model simulations
    yhat_full = calcM(Mmodel, pars, t, y0)
    
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        #np.array([d.CO2cumul]).reshape(len(d.Time),1),
                        np.array([d.Cmic]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.ATP]).reshape(len(d.Time),1)),
                     axis=1)

    #weights
    weights=np.concatenate((np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd(d.CO2cumul[0:18]).repeat(len(d.Time[0:18])).reshape(len(d.Time[0:18]),1),
                            np.nanstd((d.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.ATP)).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    out=np.nansum(((yhat_full-obs)/weights)**2)

    return out

In [ ]:
def goodnessM (x):
    #define parameters
    ##v, Km, CUE, k, kec, katp
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
        
    B_i = d.Cmicinit[0]/pars[4]
    
    y0 = np.array([S_i, B_i, 0])
    
    #times
    t = d.Time

    #model simulations
    yhat_full = calcM(Mmodel, pars, t, y0)
        
    #Standardize the simulations
    ##means
    Smeans=np.concatenate((np.nanmean(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Sstd=np.concatenate((np.nanstd(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
     

    #observations
    obs=np.concatenate((np.array([d2.S]).reshape(len(d.Time),1),
                        #np.array([d.CO2cumul]).reshape(len(d.Time),1),
                        np.array([d2.Cmic]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d2.ATP]).reshape(len(d.Time),1)),
                     axis=1)
    #Standardize the observations
    ##means
    Omeans=np.concatenate((np.nanmean(d2.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.CO2cumul[0:18]).repeat(18).reshape(18,1),
                            np.nanmean((d2.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d2.ATP).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std    
    Ostd=np.concatenate((np.nanstd(d2.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd(d.CO2cumul[0:18]).repeat(18).reshape(18,1),
                            np.nanstd((d2.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d2.ATP).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
        
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    #Normalized residual sum of squares 
    Fnorm = np.nansum((((obs-Omeans)/Ostd)-((yhat_full-Smeans)/Sstd))**2)
    
    out = np.array([R2, ll, AIC, Fnorm])

    return out

In [ ]:
def predM (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, k
    pars_model=pars[0:4]
    #conversion factors
    ##kec, katp
    conversions=pars[4:6]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total ATP, and flush (Flush)
    Flush = conversions[0]*y[:, 1]
    ATP = conversions[1]*y[:, 1]
    
    yhat = np.concatenate((y[:, 0].reshape(len(np.arange(0, 8.05, 0.05)),1),#glucose
                           #y[:, 2].reshape(len(np.arange(0, 8.05, 0.05)),1),#CO2
                           Flush.reshape(len(np.arange(0, 8.05, 0.05)),1),
                           ATP.reshape(len(np.arange(0, 8.05, 0.05)),1)), axis=1)

    return yhat

In [ ]:
j_mpars=differential_evolution(obj_funM, [(0.0001, 20), #v
                                             (0.1, 10000), #Km                                             
                                             (0, 1), #CUE
                                             (1e-12, 0.5), #k
                                             (0, 1), #kec
                                             (0, 1)]) #katp
                                             #(0, 1)]) #iX1

In [ ]:
print(j_mpars)
print(goodnessM(j_mpars.x))

In [ ]:
#initial conditions
S_i = d.Sinit[0]
    
B_i = d.Cmicinit[0]/j_mpars.x[4]
    
y0 = np.array([S_i, B_i, 0])
#times
t = np.arange(0, 8.05, 0.05)

#model simulations
Joergensen2002PredM = predM(Mmodel, j_mpars.x, t, y0)
    
#model simulations
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Joergensen2002PredM.csv', Joergensen2002PredM, delimiter=",")

## Pirt model
Model as well as supplementary functions are defined below

In [ ]:
def Pmodel (y, t, pars):
    #define initial pools
    S=y[0];    B=y[1];    CO2=y[2];
    
    #define parameters
    v=pars[0]; 
    Km=pars[1];     
    CUE=pars[2];
    m = pars[3];
    k = pars[4];
        
    #Fluxes
    uptake = v*S*B/(S + Km)
    growth = uptake*CUE
    respiration = uptake*(1 - CUE) + B*m
    death = B*k
    
    #Define derivatives
    dSdt = -uptake
    dBdt = growth - death
    dCO2dt = respiration
           
    return dSdt, dBdt, dCO2dt;

In [ ]:
def calcP (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, m, k
    pars_model=pars[0:5]
    #conversion factors
    ##kec, katp
    conversions=pars[5:7]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total ATP, and flush (Flush)
    Flush = conversions[0]*y[:, 1]
    ATP = conversions[1]*y[:, 1]
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(d.Time),1),#glucose
                           #y[:, 2].reshape(len(d.Time[0:18]),1),#CO2
                           Flush.reshape(len(d.Time),1),
                           ATP.reshape(len(d.Time),1)), axis=1)

    return yhat

In [ ]:
def obj_funP (x):
    #define parameters
    ##v, Km, CUE, m, k, kec, katp
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
        
    B_i = d.Cmicinit[0]/pars[5]
    
    y0 = np.array([S_i, B_i, 0])
    #times
    t = d.Time

    #model simulations
    yhat_full = calcP(Pmodel, pars, t, y0)
        
    #observations
    obs=np.concatenate((np.array([d.S]).reshape(len(d.Time),1),
                        #np.array([d.CO2cumul]).reshape(len(d.Time),1),
                        np.array([d.Cmic]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d.ATP]).reshape(len(d.Time),1)),
                     axis=1)

     #weights
    weights=np.concatenate((np.nanstd(d.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd(d.CO2cumul[0:18]).repeat(len(d.Time[0:18])).reshape(len(d.Time[0:18]),1),
                            np.nanstd((d.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((d.ATP)).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
       
    out=np.nansum(((yhat_full-obs)/weights)**2)

    return out

In [ ]:
def goodnessP (x):
    #define parameters
    ##v, Km, CUE, k, kec, katp
    pars = x

    #initial conditions
    S_i = d.Sinit[0]
        
    B_i = d.Cmicinit[0]/pars[5]
    
    y0 = np.array([S_i, B_i, 0])
    
    #times
    t = d.Time

    #model simulations
    yhat_full = calcP(Pmodel, pars, t, y0)
    
    #Standardize the simulations
    ##means
    Smeans=np.concatenate((np.nanmean(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    ##std
    Sstd=np.concatenate((np.nanstd(yhat_full[:, 0]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(yhat_full[:, 1]).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd((yhat_full[:, 2])).repeat(len(d.Time)).reshape(len(d.Time),1)),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(yhat_full[:, 3]).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    #observations
    obs=np.concatenate((np.array([d2.S]).reshape(len(d.Time),1),
                        #np.array([d.CO2cumul]).reshape(len(d.Time),1),
                        np.array([d2.Cmic]).reshape(len(d.Time),1),
                        #np.array([d.Cmic14]).reshape(len(d.Time),1),
                        np.array([d2.ATP]).reshape(len(d.Time),1)),
                     axis=1)
    #Standardize the observations
    ##means
    Omeans=np.concatenate((np.nanmean(d2.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.CO2cumul[0:18]).repeat(18).reshape(18,1),
                            np.nanmean((d2.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanmean(d2.ATP).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
    
    ##std    
    Ostd=np.concatenate((np.nanstd(d2.S).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanstd(d.CO2cumul[0:18]).repeat(18).reshape(18,1),
                            np.nanstd((d2.Cmic)).repeat(len(d.Time)).reshape(len(d.Time),1),
                            #np.nanmean(d.Cmic14).repeat(len(d.Time)).reshape(len(d.Time),1),
                            np.nanstd(d2.ATP).repeat(len(d.Time)).reshape(len(d.Time),1)),
                       axis=1)
        
    R2=1-np.nansum((obs-yhat_full)**2)/np.nansum((obs-np.nanmean(obs))**2)
    ll=-np.nansum((obs-yhat_full)**2)/2/np.nanstd(obs)**2
    AIC = len(pars)*2 - 2*ll
    
    #Normalized residual sum of squares 
    Fnorm = np.nansum((((obs-Omeans)/Ostd)-((yhat_full-Smeans)/Sstd))**2)
    
    out = np.array([R2, ll, AIC, Fnorm])

    return out

In [ ]:
def predP (model, pars, t, y0):
    #model parameters
    ##v, Km, CUE, m, k
    pars_model=pars[0:5]
    #conversion factors
    ##kec, katp
    conversions=pars[5:7]

    #solve the model
    y=odeint(model,y0,t, args=(pars_model,))

    #calculate total DNA, and flush (Flush)
    Flush = conversions[0]*y[:, 1]
    ATP = conversions[1]*y[:, 1]
    
    #Create data with predictions
    yhat = np.concatenate((y[:, 0].reshape(len(np.arange(0, 8.05, 0.05)),1),#glucose
                           #y[:, 2].reshape(len(np.arange(0, 8.05, 0.05)),1),#CO2
                           Flush.reshape(len(np.arange(0, 8.05, 0.05)),1),
                           ATP.reshape(len(np.arange(0, 8.05, 0.05)),1)), axis=1)

    return yhat

In [ ]:
j_ppars=differential_evolution(obj_funP, [(0.001, 500), #v
                                             (0.1, 10000), #Km                                             
                                             (0, 1), #CUE
                                             (1e-12, 0.5), #k
                                             (1e-12, 0.5), #m
                                             (0, 1), #kec
                                             (0, 1)]) #katp
                                             #(0, 1)]) #iX1

In [ ]:
print(j_ppars)
print(goodnessP(j_ppars.x))

In [ ]:
#initial conditions
S_i = d.Sinit[0]
    
B_i = d.Cmicinit[0]/j_ppars.x[5]
    
y0 = np.array([S_i, B_i, 0])

#times
t = np.arange(0, 8.05, 0.05)

#model simulations
Joergensen2002PredP = predP(Pmodel, j_ppars.x, t, y0)
    
np.savetxt('/mnt/580CBE2464C5F83D/pracovni/data_statistika/SoilMBVariability/PythonScripts/Joergensen2002PredP.csv', Joergensen2002PredP, delimiter=",")